# CNN Groceries Prediction

### Load data and transform

In [6]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder


#train and test data directory
data_dir = "data_dir/"  # Destination directory for training data
test_data_dir = "test_data_dir/"  # Destination directory for testing data


#load the train and test data
dataset = ImageFolder(data_dir,transform = transforms.Compose([
    transforms.Resize((100,100)),transforms.ToTensor()
]))
test_dataset = ImageFolder(test_data_dir,transforms.Compose([
    transforms.Resize((100,100)),transforms.ToTensor()
]))

### Creating Mini batches to make it runnable on cpu/ low ram

In [7]:
from torch.utils.data import DataLoader
# Split the dataset into smaller mini-batches using random permutation
batch_size = 32  # Original batch size
num_mini_batches = len(dataset) // batch_size  # Number of smaller mini-batches
remainder = len(dataset) % batch_size  # Remaining data

# Create a list of indices and shuffle them using random permutation
indices = torch.randperm(len(dataset))
mini_batch_indices = [indices[i:i+batch_size].tolist() for i in range(0, len(indices) - remainder, batch_size)]

# If there's remaining data, add it to the last mini-batch
if remainder > 0:
    mini_batch_indices.append(indices[-remainder:].tolist())

# Create DataLoader instances for each mini-batch
mini_batch_loaders = [DataLoader(dataset, batch_size=1, sampler=torch.utils.data.SubsetRandomSampler(indices))
                      for indices in mini_batch_indices]

### Define Model architecture

In [8]:
import torch
import torch.nn as nn

class BasicCNN(nn.Module):
    def __init__(self, num_classes):
        super(BasicCNN, self).__init__()

        # First Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Second Convolutional Layer
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Adjust the input size for the fully connected layers
        self.fc_input_size = 32 * 25 * 25  # Adjusted for 100x100 input size

        # Fully Connected Layers
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Forward pass through the first convolutional layer
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        # Forward pass through the second convolutional layer
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        # Reshape the tensor for the fully connected layers
        x = x.view(x.size(0), -1)

        # Forward pass through the fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)

        return x

# Create an instance of the BasicCNN model with the number of classes in your classification task
num_classes = 25
model = BasicCNN(num_classes)

# Print the model architecture
print(model)

BasicCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=20000, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=25, bias=True)
)


### Define Loss function, Optimizer and Hyperparameters

In [9]:
import torch.optim as optim
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10  # You can adjust the number of epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BasicCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=20000, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=25, bias=True)
)

In [ ]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for mini_batch_loader in mini_batch_loaders:
        for inputs, labels in mini_batch_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")

print("Finished Training")

### Model evaluation

In [ ]:
# Evaluate the model on the test dataset
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_dataset:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test dataset: {accuracy:.2f}%")